NOTE: first preprocess the data by running crop_images.py (for CUB-200-2011) and create_modified_datasets.ipynb. The following datastructure is required:

The following folders should be present:

	- ProtoPNet/datasets/cub200_cropped/train_cropped
	- ProtoPNet/datasets/cub200_cropped/train_cropped_contrast
	- ProtoPNet/datasets/cub200_cropped/train_cropped_saturation
    - ProtoPNet/datasets/cub200_cropped/train_cropped_hue
    - ProtoPNet/datasets/cub200_cropped/train_cropped_shape
    - ProtoPNet/datasets/cub200_cropped/train_cropped_texture
	- ProtoPNet/datasets/cub200_cropped/test_cropped
	- ProtoPNet/datasets/cub200_cropped/test_cropped_contrast
    - etc.

Each of these folders should contain a subfolder for each class containing the images
corresponding to this class. 

Then, train a ProtoPNet with main.py and the appropriate parameters in settings.py. Save the trained model.

In [ ]:
import torch
import torchvision.transforms as transforms
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
from PIL import Image

import re
import os

from preprocess import mean, std

In [ ]:
# Select model to use
load_model_dir = '../results/trained_model/'  # Model directory
load_model_name = 'trained_model.pth'# Model name of trained ProtoPNet

# Load model
load_model_path = os.path.join(load_model_dir, load_model_name)
epoch_number_str = re.search(r'\d+', load_model_name).group(0)

ppnet = torch.load(load_model_path)
ppnet = ppnet.cuda()
ppnet_multi = torch.nn.DataParallel(ppnet)

# Get network properties
img_size = ppnet_multi.module.img_size  # Image size
prototype_shape = ppnet.prototype_shape # Prototype shape
max_dist = prototype_shape[1] * prototype_shape[2] * prototype_shape[3]

# Initialize preprocessing function used for prototypes
preprocess = transforms.Compose([
   transforms.Resize((img_size,img_size)),
   transforms.ToTensor(),
   transforms.Normalize(mean=mean, std=std)
])

In [ ]:
test_dir = './datasets/cub200_cropped/train_cropped' # Path to dataset
dataset = 'train_cropped/'
# Names of the different kinds of modifications to use
modifications = ['_contrast', '_saturation', '_hue', '_shape', '_texture']

# Dataframe for storing results
results = pd.DataFrame(columns=['prototype', 'modification', 'delta'])

# Prototype indices (assumes 2000 prototypes for CUB-200-2011)
prototypes = range(2000)

# Loop through image files in all image folders
for path, subdirs, files in os.walk(os.path.join(test_dir, dataset)):
    if len(subdirs) == 0 and not "/.ipynb_checkpoints" in path:
        # Loop through files in folder
        for name in files:
            img_path = os.path.join(path, name) # Get path of file
            
            img_pil = Image.open(img_path)      # Use path to open image
            img_tensor = preprocess(img_pil)    # Apply preprocessing function, make tensor
            img_variable = Variable(img_tensor.unsqueeze(0)) # Change type

            images_test = img_variable.cuda()   # Utilize GPU

            # Get network output
            logits, min_distances = ppnet_multi(images_test)
            ref_similarities = ppnet.distance_2_similarity(min_distances)

            ref_similarities = ref_similarities[0].cpu().data.numpy()
            
            # Load the corresponding modified image and find difference
            # in similarity score for each prototype with respect to a specific
            # modification
            for modification in modifications:
                # Modify image path to get the modified image
                mod_path = img_path.replace(dataset, 
                                            dataset + modification)
                
                # Open image and convert to RGB
                try:
                    img_pil = Image.open(mod_path).convert('RGB')
                except:
                    mod_path = mod_path + '.jpg'
                    img_pil = Image.open(mod_path).convert('RGB')
                
                img_tensor = preprocess(img_pil)  # Turn image into tensor
                img_variable = Variable(img_tensor.unsqueeze(0))  # Change type

                images_test = img_variable.cuda() # Utilize GPU

                # Get network output and convert to similarity scores
                logits, min_distances = ppnet_multi(images_test)
                mod_similarities = ppnet.distance_2_similarity(min_distances)

                mod_similarities = mod_similarities[0].cpu().data.numpy() # Conversion
                delta = -1 * (mod_similarities - ref_similarities) # Get differences (per prototype)

                # Make dataframe for results (found difference)
                df = pd.DataFrame(columns=['prototype', 'modification', 'delta'])
                df['prototype'] = prototypes
                df['modification'] = modification
                df['delta'] = delta

                # Put row in total results (found differences)
                results = results.append(df, ignore_index=True)
                
        # Convert results dataframe to csv format and display
        results.to_csv('prototype_scores.csv', index=False)
        print(results)

In [ ]:
# Obtain final (global) results by calculating mean over images by prototype and modification
df_grouped = results.groupby(['prototype', 'modification']).mean()
df_grouped.to_csv('prototype_modification_results.csv') # Store results

In [ ]:
# Function to annotate a single prototype with a specific index
def annotate_prototype(index):
    annotations = {
        'Contrast': 0,
        'Saturation': 0,
        'Hue': 0,
        'Shape': 0,
        'Texture': 0
    }
    
    # Load the activation differences for the prototype
    attributes = df_grouped.loc[index]
    
    annotations['Contrast'] = attributes.loc['_contrast'][0]
    annotations['Saturation'] = attributes.loc['_saturation'][0]
    annotations['Hue'] = attributes.loc['_hue'][0]
    annotations['Shape'] = attributes.loc['_shape'][0]
    annotations['Texture'] = attributes.loc['_texture'][0]
    
    # Sort the annotations in descending order
    annotations = {k: v for k, v in sorted(annotations.items(), key=lambda item: item[1], reverse=True)}
    
    lower_than_zero = False
    i = 1

    print("Prototype " + str(index))

    # If the importance is lower than 0, leave characteristic out
    print("IMPORTANT")
    for x in annotations.items():
        if x[1] < 0 and not lower_than_zero:
            print("\nUNIMPORTANT")
            lower_than_zero = True

        print(str(i) + '. ' + x[0] + '\t (' + str(round(x[1], 5)) + ')')

        i += 1

    print('\n')
    
    return annotations

In [ ]:
# Perform annotation for all prototypes in the dataset
results = pd.DataFrame()
for i in range(2000):
    results = results.append(annotate_prototype(i), ignore_index=True)